# Vehicle Classification

## Evaluate your model


In [4]:
import os

from sklearn.metrics import accuracy_score, classification_report

from utils import  utils
from models import resnet_50

2022-06-24 09:46:25.227466: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-24 09:46:25.227584: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Experiment settings

Set here the two variables in the following way:

- **CONFIG_YML:** assign the path to the config.yml file used for the experiment you want to evaluate
- **WEIGHTS:** assign the path to the model weights (.h5 file) you want to evaluate


In [7]:
# Modify this
CONFIG_YML = "/home/matias/Models/config_exp_last.yml"
WEIGHTS = "/home/matias/Models/model.18-1.0534.h5"

In [8]:
# (!) Don't touch this cell, will load the config file specified before
config = utils.load_config(CONFIG_YML)
config['data']['directory'] = os.path.join("/home/matias/Anyone/sprint5-project",
                                           config['data']['directory'].split("/",4)[-1])
config

{'seed': 123,
 'data': {'directory': '/home/matias/Anyone/sprint5-project/data/car_ims_v2/train',
  'labels': 'inferred',
  'label_mode': 'categorical',
  'validation_split': 0.2,
  'image_size': [224, 224],
  'batch_size': 32},
 'model': {'weights': 'imagenet',
  'input_shape': [224, 224, 3],
  'classes': 196,
  'dropout_rate': 0.75,
  'l2_': 0.0001,
  'train_all': True,
  'data_aug_layer': {'random_flip': {'mode': 'horizontal_and_vertical'},
   'random_rotation': {'factor': 0.1},
   'random_zoom': {'height_factor': 0.2, 'width_factor': 0.2}}},
 'compile': {'optimizer': {'adam': {'learning_rate': 5e-05, 'beta_2': 0.999}},
  'loss': 'categorical_crossentropy',
  'metrics': ['accuracy']},
 'fit': {'epochs': 100,
  'callbacks': {'model_checkpoint': {'filepath': '/home/app/src/experiments/exp_023/model.{epoch:02d}-{val_loss:.4f}.h5',
    'save_best_only': True},
   'tensor_board': {'log_dir': '/home/app/src/experiments/exp_023/logs'}}}}

In [9]:
# (!) Don't touch this cell, will use the config file to infer the class names
#     and also to load the corresponding testing dataset.
#     If you get an error, you must review your data/code.

MODEL_CLASSES = utils.get_class_names(config)

if len(MODEL_CLASSES) != config['model']['classes']:
    raise ValueError(
        "Number of classes doesn't match between your model "
        "and your data!"
    )

_dirname, _ = os.path.split(config['data']['directory'])
TEST_FOLDER = os.path.join(_dirname, 'test')

if not os.path.exists(TEST_FOLDER):
    raise ValueError("'test' folder not found!")
    
if len(os.listdir(TEST_FOLDER)) != config['model']['classes']:
    raise ValueError(
        "Number of classes doesn't match between your model "
        "and your testing dataset!"
    )
    
if set(os.listdir(TEST_FOLDER)) != set(MODEL_CLASSES):
    raise ValueError(
        "The name of the subfolders inside your test set "
        "doesn't match with the model classes!"
    )

## Load your model

Use `resnet_50.create_model()` and remember to properly setup the model weights!

Assign the model to the variable `cnn_model`.


In [10]:
# Complete this
# TODO
cnn_model = resnet_50.create_model(WEIGHTS)

# It should print your model correctly
print(cnn_model.summary())

2022-06-24 10:32:33.493448: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-24 10:32:33.493525: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-24 10:32:33.493576: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pavilionpc): /proc/driver/nvidia/version does not exist
2022-06-24 10:32:33.868551: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 tf.__operators__.getitem (S  (None, 224, 224, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                             

## Get predictions from testing dataset

In [11]:
# (!) Don't touch this cell, will use the loaded model and
#     the function utils.predict_from_folder() to get 
#     model predictions and the corresponding true labels
#     so we can measure the accuracy

predictions, labels = utils.predict_from_folder(
    folder=TEST_FOLDER, 
    model=cnn_model, 
    input_size=config["data"]["image_size"], 
    class_names=MODEL_CLASSES,
)

if len(predictions) != len(labels):
    raise ValueError(
        "The lenght of predictions and labels lists doesn't match!"
    )

if not isinstance(predictions[0], str):
    raise ValueError(
        "Model predictions should be represented as string. E.g: 'Acura RL Sedan 2012'"
    )

if not isinstance(labels[0], str):
    raise ValueError(
        "Ground true labels should be represented as string. E.g: 'Acura RL Sedan 2012'"
    )


2022-06-24 10:48:30.995208: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 271934520 exceeds 10% of free system memory.


In [12]:
# (!) Don't touch this cell

print(classification_report(y_true=labels, y_pred=predictions))


                                                        precision    recall  f1-score   support

                            AM General Hummer SUV 2000       0.69      0.66      0.67        44
                             Acura Integra Type R 2001       0.57      0.73      0.64        44
                                   Acura RL Sedan 2012       1.00      0.03      0.06        32
                                   Acura TL Sedan 2012       0.51      0.79      0.62        43
                                  Acura TL Type-S 2008       0.32      0.21      0.26        42
                                  Acura TSX Sedan 2012       0.49      0.47      0.48        40
                              Acura ZDX Hatchback 2012       0.28      0.49      0.36        39
              Aston Martin V8 Vantage Convertible 2012       0.40      0.31      0.35        45
                    Aston Martin V8 Vantage Coupe 2012       0.43      0.07      0.12        41
                  Aston Martin Virage C

In [13]:
# (!) Don't touch this cell

acc = accuracy_score(y_true=labels, y_pred=predictions)

print(f"Your model accuracy is {acc:.4f}!")

if acc < .3:
    raise ValueError("Your model accuracy is too low :(\nYou can do it better! :)")


Your model accuracy is 0.4308!


### Experiments
After I made severals experiments I got the next conclutions:
* SGD as optimizer was like more sensible to changes and performed worst in this project
* The model tend to overfit so I chose the weigth from the epoch 18
* If you keep the model training for more epochs the accuracy on the test data improves but at the cost that now we have huge gap with the training data accuracy
* l2 as regularization worked better than dropuot but to much undermine the performance of the model (1e-2 <= l2 <= 1e-4)  
* With freezed parameters the models couldn't surpass the 25-30% of accuracy
* With cropped images there was a gain of 5-10% of accuracy (with frezzed conv. layers)

#### Results  
Main Parameters:  
  

l2 = 1e^{-4}  
dropout rate = 0.75  
train all = True  
optimizer = adam | learning rate = 5e-05, beta 2 = 0.999 |  
images dataset = Cropped images
   
Epoch 18:  


Accuracy train:0.7638  
Accuracy validation:0.7082  
Accuracy test:0.4308  
Loss train:0.8681  
Loss validation:1.053  
